## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？

简单的TensorFlow可以简单替代 NumPy，但复杂级别的项目还是要专门的TensorFlow。
主要区别：
– 随机数策略不同（tf 有全局/局部随机生成器，np 用 RandomState）；
– 整数除法默认行为不同（tf 和 Python3 一致，np 默认截断除法）；
– 布尔型索引写法不同（tf 用 tf.boolean_mask）；
– 可变长度循环需用 tf.while_loop/tf.scan

2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？

形状、dtype、数值都一样，但：
• tf.range 在图里用 Range 算子，不占用内存常量区；
• tf.constant(np.arange(10)) 会把 10 个整数拷进图常量，图文件会大 10×4/8 B。
tf.range(10)：
<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>
tf.constant(np.arange(10))：
<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？

• 函数：
– 仅依赖 y_true/y_pred（以及外部闭包里可序列化的超参）；
– 无需保存配置，一行写完；
– 典型场景：交叉熵＋权重、Focal loss、Dice loss。
• 子类：
– 需要保存/恢复超参（如 α、γ、margin）；
– 想在 .compile() 里只传类名，让 Keras 自动实例化。

4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？

• 函数：
– 每个 batch 返回一个标量即可，不跨批次累加；
– 适合“一次性”指标，如当前 batch 的 top-3 accuracy。
• 子类：
– 必须跨批次累积（accuracy、AUC、F1）；
– 需要 .update_state()、.result()、.reset_states()；
– 希望 model.fit 自动调用。

5. 什么时候应该自定义层而不是自定义模型？

Layer = “张量 → 张量”的可复用积木，无外部数据依赖，可放入任意模型；
Model = 端到端有输入/输出的完整图，可 .fit/.predict/.save。

6. 有哪些示例需要编写自定义训练循环？

对抗训练（GAN、FGSM、PGD）——需要交替更新两组网络；
强化学习——每步环境交互后才产生标签，批次大小不一；
知识蒸馏——教师模型输出当软标签，需要中间特征对齐；
多任务/多阶段——不同任务用不同优化器、学习率、梯度裁剪；
梯度累积、混合精度、梯度检查点等需手动控制 tf.GradientTape()；
非标准日志（需要把每步的梯度范数、参数直方图写回 TensorBoard）

7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？

• 能放，但只在 eager 下永远 OK；
• 一旦 @tf.function 装饰（或 model.fit 默认 autograph），必须能被 AutoGraph 识别/转换，否则：
– 报错（py 内置 list 插入、非 tf 控制流、外部 I/O）；
– 或 silently 回退到 py func，图断裂，性能暴跌。
写组件时，把“训练相关计算”全用 tf 原生算子

8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？

• 在 tf.function 内修改外部 Python list/dict 等非 tf.Variable；
• 用 Python 的 for/while 遍历 Tensor 第一维（应改用 tf.range+tf.while_loop）；
• 用 if tensor: 判断（应 if tf.reduce_sum(tensor) > 0:）；
• 在图里放 print()/time.sleep()/input() 等阻塞 I/O；
• 用 numpy() 把张量拖回 CPU（会触发图外拷贝，直接报错）；
• 依赖全局随机状态（np.random、random模块），应改用 tf.random.* 并传入 seed。

9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？

• 输入序列长度每次都变，且后续层要用到该长度（如 Transformer 的 mask）；
• 每次训练样本数量不同，或需要动态 RNN 展开；
• 网络拓扑本身随输入而变（动态点云、图神经网络、Neural Architecture Search 一次采一个子图）。


## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。

2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [116]:
# Dense层的简化自定义实现  # 形状 输出的长度，激活函数， W， b
import tensorflow as tf

class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf,keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": tf.keras.activations.serialize(self.activation)}

In [127]:
# 执行层归一化的自定义层

import tensorflow as tf

class LayerNormalization(tf.keras.layers.Layer):
    """
    逐样本、逐特征维度的 Layer Normalization 自定义层。
    公式:
        y = α ⊙ (x - μ) / (σ + ε) + β
    """

    def __init__(self, epsilon: float = 1e-3, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, batch_input_shape):
        """
        定义两个可训练权重：
           α 初始化为 1
           β 初始化为 0
           形状均为 input_shape[-1:]
        """
        self.alpha = self.add_weight(
            name='alpha',
            shape=batch_input_shape[-1:],
            initializer='ones')
        self.beta = self.add_weight(
            name='beta',
            shape=batch_input_shape[-1:],
            initializer='zeros')
        super().build(batch_input_shape)

    def call(self, X):
        """
        计算每个实例特征的均值与标准差，并返回归一化结果
        """
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / tf.sqrt(variance + self.epsilon) + self.beta

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'epsilon': self.epsilon}

In [118]:
x = tf.random.normal((64,64,64))
offical_LayerNormalization = tf.keras.layers.LayerNormalization(epsilon=1e-3)
own_LayerNormalization = LayerNormalization(epsilon=1e-3)

y1 = offical_LayerNormalization(x, training=False)
y2 = own_LayerNormalization(x, training=False)

diff = tf.reduce_max(y1 - y2)
print("max_diff", tf.abs(diff).numpy())

max_diff 7.1525574e-07


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

tf.random.set_seed(42)
np.random.seed(42)


In [137]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = tf.cast(x_train, tf.float32) / 255.0
x_test  = tf.cast(x_test,  tf.float32) / 255.0
x_train = tf.expand_dims(x_train, -1)   # (60000, 28, 28, 1)
x_test  = tf.expand_dims(x_test,  -1)

In [138]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
          .shuffle(60000).batch(128).prefetch(tf.data.AUTOTUNE)
val_ds   = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
          .batch(128).prefetch(tf.data.AUTOTUNE)

In [139]:
model = tf.keras.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(512, name="dense1"),
    LayerNormalization(),
    layers.ReLU(),
    layers.Dense(256, name="dense2"),
    LayerNormalization(),
    layers.ReLU(),
    layers.Dense(10, name="logits")
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [140]:
bottom_vars, top_vars = [], []
for layer in model.layers:
    if layer.name in {"dense1", "dense2"} or isinstance(layer, LayerNormalization):
        bottom_vars += layer.trainable_variables
    elif layer.name == "logits":
        top_vars += layer.trainable_variables

opt_bottom = tf.keras.optimizers.Adam(1e-4)
opt_top    = tf.keras.optimizers.SGD(1e-2)

In [141]:
@tf.function
def train_step(x, y, train_loss, train_acc):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss   = loss_fn(y, logits)
    grads = tape.gradient(loss, bottom_vars + top_vars)
    g_bottom, g_top = grads[:len(bottom_vars)], grads[len(bottom_vars):]
    opt_bottom.apply_gradients(zip(g_bottom, bottom_vars))
    opt_top.apply_gradients(zip(g_top, top_vars))
    train_loss.update_state(loss)
    train_acc.update_state(y, logits)

@tf.function
def val_step(x, y, val_loss, val_acc):
    logits = model(x, training=False)
    loss   = loss_fn(y, logits)
    val_loss.update_state(loss)
    val_acc.update_state(y, logits)

In [142]:
EPOCHS = 20
for epoch in range(1, EPOCHS + 1):
    # ★★★ 每 epoch 新建一套指标 ★★★
    train_loss = tf.keras.metrics.Mean()
    train_acc  = tf.keras.metrics.SparseCategoricalAccuracy()
    val_loss   = tf.keras.metrics.Mean()
    val_acc    = tf.keras.metrics.SparseCategoricalAccuracy()

    # 训练
    for x, y in train_ds:
        train_step(x, y, train_loss, train_acc)

    # 验证
    for x, y in val_ds:
        val_step(x, y, val_loss, val_acc)

    print(f"Epoch {epoch}  "
          f"train_loss {train_loss.result():.4f}  train_acc {train_acc.result():.4f}  "
          f"val_loss {val_loss.result():.4f}  val_acc {val_acc.result():.4f}")


Epoch 1  train_loss 0.4067  train_acc 0.8871  val_loss 0.1806  val_acc 0.9495
Epoch 2  train_loss 0.1479  train_acc 0.9589  val_loss 0.1270  val_acc 0.9623
Epoch 3  train_loss 0.1016  train_acc 0.9723  val_loss 0.1002  val_acc 0.9705
Epoch 4  train_loss 0.0757  train_acc 0.9801  val_loss 0.0881  val_acc 0.9739
Epoch 5  train_loss 0.0575  train_acc 0.9857  val_loss 0.0794  val_acc 0.9756
Epoch 6  train_loss 0.0449  train_acc 0.9899  val_loss 0.0721  val_acc 0.9777
Epoch 7  train_loss 0.0350  train_acc 0.9927  val_loss 0.0709  val_acc 0.9773
Epoch 8  train_loss 0.0271  train_acc 0.9954  val_loss 0.0686  val_acc 0.9778
Epoch 9  train_loss 0.0213  train_acc 0.9967  val_loss 0.0642  val_acc 0.9788
Epoch 10  train_loss 0.0165  train_acc 0.9981  val_loss 0.0629  val_acc 0.9799
Epoch 11  train_loss 0.0127  train_acc 0.9988  val_loss 0.0619  val_acc 0.9792
Epoch 12  train_loss 0.0100  train_acc 0.9993  val_loss 0.0630  val_acc 0.9806
Epoch 13  train_loss 0.0080  train_acc 0.9995  val_loss 0.061

In [4]:
tf.range(10)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [5]:
tf.constant(np.arange(10))

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>